In [23]:
from nautilus_trader.model.enums import BookAction
from nautilus_trader.model.enums import BookType
from nautilus_trader.model.orderbook.data import OrderBookDeltas
from nautilus_trader.model.orderbook.data import Order
from nautilus_trader.model.orderbook.data import OrderBookDelta
from nautilus_trader.model.orderbook.book import OrderBook
# from nautilus_trader.model.orderbook.book import L2OrderBook
from nautilus_trader.model.identifiers import InstrumentId, Symbol, Venue
from nautilus_trader.backtest.data.providers import TestInstrumentProvider
from collections import defaultdict
from nautilus_trader.examples.strategies.market_maker import MarketMaker
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money
from nautilus_trader.model.enums import OMSType
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.currencies import USD, USDT, BTC
from nautilus_trader.model.objects import Price, Quantity
from nautilus_trader.backtest.node import BacktestNode, BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.config.common import ImportableStrategyConfig, StrategyConfig

from decimal import Decimal

import pandas as pd

In [24]:
bin_btcusdt_ticks_file = ''
# data_file = '../../data/l2_data_extract_discord/BINANCE-BTC-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz'

data_file = '/Users/andrewgoldberg/Projects/hummingbot-backtest/data/l2_data_extract_discord/BINANCE-BTC-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz'
# data_df = pd.read_csv(data_file, chunksize=100000, compression='gzip', error_bad_lines=False)
# data_df = pd.read_csv(data_file, nrows=100000, compression='gzip', error_bad_lines=False)

In [25]:
import os, shutil
CATALOG_PATH = os.getcwd() + "/catalog"

# Clear if it already exists, then create fresh
# if os.path.exists(CATALOG_PATH):
#     shutil.rmtree(CATALOG_PATH)
# os.mkdir(CATALOG_PATH)

from nautilus_trader.persistence.catalog import ParquetDataCatalog
catalog = ParquetDataCatalog(CATALOG_PATH)

from nautilus_trader.persistence.external.core import process_files, write_objects
from nautilus_trader.backtest.data.providers import TestInstrumentProvider

BIN_BTCUSDT = TestInstrumentProvider.btcusdt_binance()

from nautilus_trader.persistence.external.core import write_objects

# write_objects(catalog, [BIN_BTCUSDT])

catalog.instruments()


,id,native_symbol,base_currency,quote_currency,price_precision,size_precision,price_increment,size_increment,lot_size,max_quantity,...,min_notional,max_price,min_price,margin_init,margin_maint,maker_fee,taker_fee,info,ts_event,ts_init
0,BTCUSDT.BINANCE,BTCUSDT,BTC,USDT,2,6,0.01,0.000001,NaN,9000.000000,...,10.00000000 USDT,1000000.00,0.01,0,0,0.001,0.001,None,0,0


In [26]:
catalog.list_data_types()

['currency_pair', 'order_book_data']

In [27]:
BIN_BTCUSDT

CurrencyPair(id=BTCUSDT.BINANCE, native_symbol=BTCUSDT, asset_class=CRYPTO, asset_type=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)

In [28]:
BIN_BTCUSDT.venue

Venue('BINANCE')

In [29]:
StrategyConfig()

StrategyConfig(strategy_id=None, order_id_tag=None, oms_type=None)

In [50]:
# print('start')
# Configure backtest engine
# config = BacktestEngineConfig(
#     trader_id="BACKTESTER-001",
# )
# print('config')
# Build the backtest engine
# engine = BacktestEngine(config=config)
# print('set config')
# Add a trading venue (multiple venues possible)
venues_config=[
    BacktestVenueConfig(
        name="BINANCE",
        oms_type="NETTING",
        account_type="CASH",
        book_type="L2_MBP",
        # base_currency='BTC',  # Standard single-currency account
        starting_balances=["10000 BTC", "100000 USDT"],  # Single-currency or multi-currency accounts
    )
]
data_config=[
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=OrderBookDeltas,
        instrument_id=BIN_BTCUSDT.id.value
    )
]
strategy = MarketMaker
strategy_config = ImportableStrategyConfig(
    strategy_path=f"{strategy.__module__}:{strategy.__name__}",
    config_path=f"{strategy.__module__}:{strategy.__name__}Config",
    config=dict(
        instrument_id = str(BIN_BTCUSDT.id.value),
        trade_size = Decimal(0.003),
        max_size = Decimal(400_000_000),
    ),
),

# strategy = MarketMaker(
#     instrument_id = BIN_BTCUSDT.id,
#     trade_size = Decimal(0.003),
#     max_size = Decimal(400_000_000),
# )
# strategies = [strategy]
engine_config = BacktestEngineConfig(
    stratregies=[strategy_config],
    run_analysis=True
)
config = BacktestRunConfig(
    engine=engine_config,
    data=data_config,
    venues=venues_config,
)




# # Run the engine (from start to end of data)
# engine.run_streaming()
# print('run streaming')
# # Optionally view reports
# with pd.option_context(
#     "display.max_rows",
#     100,
#     "display.max_columns",
#     None,
#     "display.width",
#     300,
# ):
#     print(engine.trader.generate_account_report(BIN_BTCUSDT))
#     print(engine.trader.generate_order_fills_report())
#     print(engine.trader.generate_positions_report())

# # For repeated backtest runs make sure to reset the engine
# engine.reset()

# # Good practice to dispose of the object when done
# engine.dispose()

In [51]:

node.configs

[BacktestRunConfig(engine=BacktestEngineConfig(environment=<Environment.BACKTEST: 'backtest'>, trader_id='BACKTESTER-001', cache=None, cache_database=None, data_engine=DataEngineConfig(debug=False), risk_engine=RiskEngineConfig(bypass=False, max_order_rate='100/00:00:01', max_notional_per_order={}, debug=False), exec_engine=ExecEngineConfig(load_cache=True, allow_cash_positions=True, debug=False), streaming=None, actors=[], strategies=[], load_state=False, save_state=False, loop_debug=False, log_level='INFO', bypass_logging=False, run_analysis=True), venues=[BacktestVenueConfig(name='BINANCE', oms_type='NETTING', account_type='CASH', starting_balances=['10000 BTC', '100000 USDT'], base_currency=None, default_leverage=1.0, leverages=None, book_type='L2_MBP', routing=False, frozen_account=False, reject_stop_orders=True)], data=[BacktestDataConfig(catalog_path='/Users/andrewgoldberg/opt/anaconda3/envs/web3/lib/python3.9/site-packages/nautilus_trader/examples/catalog', data_cls='nautilus_t

In [52]:
node = BacktestNode(configs=[config])

results = node.run()